# Navigation of the robot


In [87]:
import tdmclient.notebook
from Navigation_utils import *
await tdmclient.notebook.start()

In [88]:
%%run_python

# Let's intitialize the variables

speed0 = 100       #initial speed
speed_gain = 2     # 
obstThrL = 10      # Low threshold for which the robot exits the avoidance state
obstThrH = 20      # High threshold for which the robot enters the avoidance state
obstSpeedGain = 5  # gain associated to obstacle avoidance 
close_obst = 0     # number of sensors that detect an obstacle

state = 0          # 0 = normal state, 1 = obstacle avoidance state
path_reached = 0
prox = [0,0,0,0,0,0,0] # obstacle detection array
obj_right = 0      # 1 = object detected on the right, 0 = object detected on the left

button_center_last = 0
timer_period[0] = 10
'''
@onevent
def button_center():
    motor_right=0
    motor_left=0
'''
@onevent
def button_center():  # J'ai créé ca pour simuler le fait que j'ai réatteint ton chemin en appuyant sur le bouton, mais en vrai ca sera avec une condition quand j'aurai tes segments et la position
    global path_reached, button_center_last

    if button_center and not button_center_last: 
        if (path_reached == 0):
            path_reached = 1
        else:
            path_reached = 0
        print(path_reached)

    button_center_last = button_center

@onevent
def timer0():
    global speed0, speed_gain, obstThrL, obstThrH, obstSpeedGain, state, prox, motor_left_target, motor_right_target, leds_top, prox_horizontal, path_reached, close_obst, obj_right

    prox = prox_horizontal
    if state == 0:
        for i in range (6):
            if prox[i] > obstThrH:
                state = 1
    else:
        close_obst = 0          # number of sensors that detect an obstacle
        for i in range (6):
            if prox[i] > obstThrL:
                close_obst = close_obst + 1
                if i>2:
                    obj_right = 1
                else:
                    obj_right = 0
        if close_obst == 0:
            if path_reached == 1:
                state = 0

    actuation = [0,0]
    if state == 0:
        #global_navigation()
        leds_top = [0,0,0]
        motor_left_target = speed0
        motor_right_target = speed0
    else: 
        #obstacle_surrounding()  
        leds_top = [30,30,30]
        w_l = [40,  20, -20, -20, -40,  30, -10]
        w_r = [-40, -20, -20,  20,  40, -10,  30]
        x = [0,0,0,0,0,0,0]
        sensor_translation = 100

        if close_obst == 0:
            if (obj_right == 1):
                actuation[0] = 50
                actuation[1] = -25
            else:
                actuation[0] = -25
                actuation[1] = 50
        else:
            for i in range (6):
                x[i] = prox[i] // sensor_translation
                actuation[0] = actuation[0] + x[i] * w_l[i]
                actuation[1] = actuation[1] + x[i] * w_r[i]

        motor_left_target = speed0 + actuation[0]
        motor_right_target = speed0 + actuation[1]





In [89]:
await tdmclient.notebook.stop()